In [20]:
def read_pgm(pgmf):
    rs = pgmf.readline()
    rd = pgmf.readline()
    if str(rd).find('#') != -1 :
        rd = pgmf.readline()
    (width, height) = [int(i) for i in rd.split()]
    depth = int(pgmf.readline())

    raster = []
    for y in range(height):
        row = []
        for x in range(width):
            row.append(ord(pgmf.read(1)))
        raster.append(row)
    return raster

In [36]:
def writePGM(file, w, h, maxi, flatImg):
    file.write('P5\n'.encode())
    file.write('{} {}\n'.format(w, h).encode())
    file.write('{}\n'.format(maxi).encode())
    file.write(bytearray(flatImg))
    file.close()

In [48]:
pic_1 = open("D:/Project/DIP/HW1/4/distgrid.pgm","rb")
pic_1n = read_pgm(pic_1)
pic_2 = open("D:/Project/DIP/HW1/4/distlenna.pgm","rb")
pic_2n = read_pgm(pic_2)
pic_1.close()
pic_2.close()

Import

In [23]:
from numpy.linalg import solve

Def

In [24]:
class Segment:
    def __init__(self,xs,ys):
        self.xs = xs
        self.ys = ys
    
def ExpectedGrid():
    segments = []
    for y in range(0,256,16):
        for x in range(0,256,16):
            xp = x + 16 if x + 16 < 256 else 255
            yp = y + 16 if y + 16 < 256 else 255
            xs = [x,xp,x,xp]
            ys = [y,y,yp,yp]
            segments.append(Segment(xs,ys))
    return segments

def DistGrid():
    segments = []
    file = open('./text.txt','r')
    for _ in range(256):
        xs = [int(x) for x in file.readline().split()]
        ys = [int(y) for y in file.readline().split()]
        segments.append(Segment(xs,ys))
    return segments

In [25]:
def tI(num):
    return 1 if num%1 >= 0.5 else  0

In [26]:
def Interpolation(n, x, y):
    x = tI(x);y = tI(y)
    return n[y][x]

In [27]:
def fillImage(oriimage, image):
    d_Grid = DistGrid()
    e_Grid = ExpectedGrid()
    [fillSegment(d_Grid[i], e_Grid[i], oriimage, image) for i in range(len(d_Grid))]

In [28]:
def fillSegment(d_Segment, e_Segment, d_image, image):
    ex = e_Segment.xs;ey = e_Segment.ys;dx = d_Segment.xs;dy = d_Segment.ys
    abcd, efgh = find_Linear(ex, ey, dx, dy)
    for y in range(int(ey[0]), int(ey[-1])):
        for x in  range(int(ex[0]), int(ex[-1])):
            p_y = find_coordinate(abcd, x, y)
            p_x = find_coordinate(efgh, x, y)
            _00_, _01_, _10_ ,_11_ = Neighbor(int(p_x), int(p_y), d_image)
            image[y][x] = Interpolation([[_00_, _01_],[_10_, _11_]], p_x, p_y)

In [29]:
def Neighbor(i, j, image):
    return image[i][j],(image[i][j+1] if j + 1 < 256 else 0),(image[i+1][j] if i + 1 < 256 else 0),(image[i+1][j+1] if j +1 < 256 and i +1 < 256 else 0)

In [30]:
def find_Linear( ex_x, ex_y,d_x, d_y ):
    origin = [[ ex_x[i], ex_y[i], ex_x[i]*ex_y[i], 1] for i in range(-1,-5,-1)]
    x = [d_x[i] for i in range(-1, -5, -1)]
    y = [d_y[i] for i in range(-1, -5, -1)]
    return solve(origin,x) , solve(origin,y)

In [31]:
def find_coordinate( co, x, y):
    a, b, c, d = co
    return a*x + b*y + c*x*y + d 

In [32]:
def flatten(arr):
    return sum(arr, [])

Run

In [44]:
draft = [[0]*len(pic_1n[0]) for i in range(len(pic_1n))]
fillImage(pic_1n, draft)

In [45]:
save_dir = open("./4/out_grid.pgm","wb")
writePGM(save_dir, len(pic_1n[0]) ,len(pic_1n), 255 , flatten(draft))

In [49]:
draft_2 = [[0]*len(pic_2n[0]) for i in range(len(pic_2n))]
fillImage(pic_2n, draft_2)

In [50]:
save_dir = open("./4/out_lenna.pgm","wb")
writePGM(save_dir, len(pic_1n[0]) ,len(pic_1n), 255 , flatten(draft_2))